In [1]:
import datetime
import openai
import json
import inspect
import glob

from llmcam.ytlive import capture_youtube_live_frame
from llmcam.gpt4v import ask_gpt4v
from typing import Optional

In [2]:
def capture_youtube_live_frame_and_save() -> str:
    "Capture a jpeg file from YouTube Live and save in data directory"
    return str(capture_youtube_live_frame())

In [3]:
def ask_gpt4v_about_image_file(path:str) -> str:
    "Tell all about quantitative information from a given image file"
    info = ask_gpt4v(path)
    return json.dumps(info)

In [4]:
def tool_schema(func):
    """ automatically generate a schema from its parameters and docstring"""
    # Extract function name, docstring, and parameters
    func_name = func.__name__
    func_description = func.__doc__ or "No description provided."
    signature = inspect.signature(func)
    
    # Create parameters schema
    parameters_schema = {
        "type": "object",
        "properties": {},
        "required": []
    }
    
    # Populate properties and required fields
    for param_name, param in signature.parameters.items():
        param_type = param.annotation if param.annotation != inspect._empty else str
        # Map Python types to JSON Schema types
        json_type = "string" if param_type == str else "number" if param_type in [int, float] else "boolean"
        
        # Add parameter to schema
        parameters_schema["properties"][param_name] = {
            "type": json_type,
            "description": f"{param_name} parameter of type {param_type.__name__}"
        }
        
        # Mark as required if no default
        if param.default == inspect.Parameter.empty:
            parameters_schema["required"].append(param_name)
    
    # Build final tool schema
    tool_schema = [
        {
            "type": "function",
            "function": {
                "name": func_name,
                "description": func_description,
                "parameters": parameters_schema,
            }
        }
    ]
    return tool_schema

In [5]:
# Environmental setting up
tools = tool_schema(capture_youtube_live_frame_and_save) + tool_schema(ask_gpt4v_about_image_file)
messages = [{"role":"system", "content":"You are a helpful system administrator. Use the supplied tools to assist the user."}]

In [6]:
# Support functions to handle tool response,where res == response.choices[0].message
def fn_name(res): return res.tool_calls[0].function.name
def fn_args(res): return json.loads(res.tool_calls[0].function.arguments)    
def fn_exec(res): return globals().get(fn_name(res))(**fn_args(res))
def fn_result_content(res):
    """Create a content containing the result of the function call"""
    content = dict()
    content.update(fn_args(res))
    content.update({fn_name(res): fn_exec(res)})
    return json.dumps(content)

In [7]:
def complete(role, content, tool_call_id=None, printing=False):
    "Send completion request with messages, and save the response in messages again"
    messages.append({"role":role, "content":content, "tool_call_id":tool_call_id})
    response = openai.chat.completions.create(model="gpt-4o", messages=messages, tools=tools)
    res = response.choices[0].message
    messages.append(res.to_dict())
    if res.to_dict().get('tool_calls'):
        complete(role="tool", content=fn_result_content(res), tool_call_id=res.tool_calls[0].id)
    if printing==True:
        for i, m in enumerate(messages):
            print(i, m)

In [8]:
complete("user", "Hi, can you capture YouTube Live?", printing=True)

0 {'role': 'system', 'content': 'You are a helpful system administrator. Use the supplied tools to assist the user.'}
1 {'role': 'user', 'content': 'Hi, can you capture YouTube Live?', 'tool_call_id': None}
2 {'content': 'Yes, I can capture a frame from a YouTube Live stream and save it for you. Would you like me to do that now?', 'refusal': None, 'role': 'assistant'}


In [9]:
complete("user", "Yes, please", printing=True)

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading web creator player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
31.10.2024 21:38:15 Tuomiokirkko
0 {'role': 'system', 'content': 'You are a helpful system administrator. Use the supplied tools to assist the user.'}
1 {'role': 'user', 'content': 'Hi, can you capture YouTube Live?', 'tool_call_id': None}
2 {'content': 'Yes, I can capture a frame from a YouTube Live stream and save it for you. Would you like me to do that now?', 'refusal': None, 'role': 'assistant'}
3 {'role': 'user', 'content': 'Yes, please', 'tool_call_id': None}
4 {'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_PBOAVvUB9xuFi1vK5Fg8tAcx', 'function': {'arguments': '{}', 'name': 'capture_youtube_live_frame_and_save'}, 'type': 'function'}]}
5 {'role': 'tool', 'content': '

In [10]:
files = sorted(glob.glob("./data/cap_*.jpg"))
len(files), files[:3]

(496,
 ['./data/cap_2024.09.28_15:59:06_Presidentinlinna.jpg',
  './data/cap_2024.09.28_16:00:11_Presidentinlinna.jpg',
  './data/cap_2024.09.28_16:01:16_Etelasatama.jpg'])

In [11]:
complete("user", f"Hi, can you tell any infomation about {files[-1]}?", printing=True) 

0 {'role': 'system', 'content': 'You are a helpful system administrator. Use the supplied tools to assist the user.'}
1 {'role': 'user', 'content': 'Hi, can you capture YouTube Live?', 'tool_call_id': None}
2 {'content': 'Yes, I can capture a frame from a YouTube Live stream and save it for you. Would you like me to do that now?', 'refusal': None, 'role': 'assistant'}
3 {'role': 'user', 'content': 'Yes, please', 'tool_call_id': None}
4 {'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_PBOAVvUB9xuFi1vK5Fg8tAcx', 'function': {'arguments': '{}', 'name': 'capture_youtube_live_frame_and_save'}, 'type': 'function'}]}
5 {'role': 'tool', 'content': '{"capture_youtube_live_frame_and_save": "../data/cap_2024.10.31_21:38:15_Tuomiokirkko.jpg"}', 'tool_call_id': 'call_PBOAVvUB9xuFi1vK5Fg8tAcx'}
6 {'content': 'I have captured a frame from the YouTube Live stream and saved it as "cap_2024.10.31_21:38:15_Tuomiokirkko.jpg". Is there anything specific you\'d like to kno

In [13]:
print(messages[-1]['content'])

The image "cap_2024.10.31_11:52:57_Valkosaari.jpg" captured at Valkosaari on October 31, 2024, has the following details:

- **Dimensions:** 1280 x 720 pixels
- **Timestamp:** 2024-10-31T11:52:57
- **Location:** Valkosaari
- **Buildings:** There are 5 buildings visible, with heights ranging from 1 to 2 stories.
- **Water Bodies:** A harbor is visible with 3 boats present.
- **Sky:** The sky is visible, and it appears to be a daytime scene.

If there's anything else you need to know or do with this image, feel free to ask!
